In [ ]:
## kaggle code
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## installations
#!pip install turicreate
        
## all imports 
import numpy as np 
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import AdaBoostClassifier
import turicreate as tc

        
        
# load the data 
train = pd.read_csv('../input/bgutreatmentoutcome/train.CSV')
samplesub = pd.read_csv('../input/bgutreatmentoutcome/sampleSubmission.CSV')
test = pd.read_csv('../input/bgutreatmentoutcome/test.CSV')

train_empty_columns = train[train.columns[train.isna().all()]].columns
test_empty_columns = test[test.columns[test.isna().all()]].columns
empty_columns = list(set(test_empty_columns)|set(train_empty_columns))


def preprocessing(df, columns_to_remove):
    string_columns_index = [i for i in range(83)]
    string_columns_names = ['A' + str(i) for i in range(1,84)] 
    numrical_columns  = [i for i in range(83,452)]
    numrical_columns_names  = ['A' + str(i) for i in range(84,453)]
    categorial_columns = [x for x in string_columns_names if x not in columns_to_remove]
    
    ## cleaning the data
    #filling NA
    
    df[string_columns_names] = df[string_columns_names].fillna(df[string_columns_names].mode().iloc[0])
    df[numrical_columns_names] = df[numrical_columns_names].fillna(df[numrical_columns_names].mean())
    df.drop(columns_to_remove, inplace = True, axis = 1)

# Feature engineering

    df_ohe = pd.get_dummies(df,prefix = categorial_columns, columns = categorial_columns)
    return df_ohe

##
train_processed = preprocessing(train, empty_columns)
features = list(train_processed.columns)
features.remove('CLASS')
train_processed = train_processed[features +['CLASS']]
test_processed = preprocessing(test, empty_columns)

train_processed,test_processed =  train_processed.align(test_processed, join = 'left', axis = 1)
test_processed.drop('CLASS', inplace = True, axis = 1)

test_processed = test_processed.fillna(0)


## Boosted tree by turicreate
train_sf = tc.SFrame(train_processed)
test_sf = tc.SFrame(test_processed)
cls = tc.boosted_trees_classifier.create(train_sf,features=features, target="CLASS", random_seed = 42)
cls.evaluate(train_sf)
prob = cls.predict(test_sf, output_type='probability')
df = pd.DataFrame({"id" : [x for x in range(1, len(prob)+1)],"ProbToYes" : list(prob.to_numpy())})

##Adaboost by sklearn
clf = AdaBoostClassifier(n_estimators = 1000, random_state = 42)
clf.fit(train_processed.iloc[:, :-1], train_processed.iloc[:,-1])
clf.score(train_processed.iloc[:, :-1], train_processed.iloc[:,-1])

prob2 = clf.predict_proba(test_processed)

df2 = pd.DataFrame({"id" : [x for x in range(1, len(prob)+1)],"ProbToYes" : list(prob2[:,1])})
df2.to_csv('sub2.csv', index = None)

